In [ ]:
# Imports for the project

#!pip install LogUniform SpeedTorch cupy

import os
import shutil
import datetime

from azureml.core.workspace import Workspace
from azureml.core import Experiment

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.dnn import PyTorch

In [ ]:
# Set up the workspace and config for the project

# Subscription_id is my personal id - change to your own if you're running this on a different account

sub_id = '' 
name = 'Azure-AI-Hackathon-Proj'
res_group = name


ws = Workspace.get(name=name,
                       subscription_id=sub_id,
                       resource_group=res_group)


In [ ]:
# Create the source folder and the experiment
project_folder = './pytorch-lit2vec2'
os.makedirs(project_folder, exist_ok=True)

experiment_name = 'pytorch-lit2vec2'
experiment = Experiment(ws, name=experiment_name)

In [ ]:
# Make sure the training script and all other import files are uploaded to the experiment folder

shutil.copy('lit2vec2training_start9_9_19.py', project_folder)


In [ ]:
# Allocate the compute resources for this job

cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

In [ ]:
# Change as necessary.
# AFAIK no params are needed for this script, but could modify so everything is controlled from this notebook 
script_params = {}



# Varaiable to fill with the packages needed to train this model. 
# Installing packages this way can be quite messy. 
# Using workaround by installing within training script.
pkgs = []
#['LogUniform', 'SpeedTorch', 'cupy']


estimator = PyTorch(source_directory=project_folder, 
                    script_params=script_params,
                    compute_target=compute_target,
                    entry_script='lit2vec2training_start9_9_19.py',
                    use_gpu=True,
                    pip_packages=pkgs)

In [ ]:
# Submit run of the experiment to Azure workspace

run = experiment.submit(estimator)
run.wait_for_completion(show_output=True)

In [ ]:
# Register the model to the outputs folder
os.makedirs('outputs/', exist_ok=True)

model = run.register_model(model_name='pt-l2v2', model_path='outputs/')



In [ ]:
# When running on local machine, dowload using this cell
'''
# Create a model folder in the current directory

os.makedirs('./model', exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join('./model', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)
        
'''
